# Fit data with a linear least squares method

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

LikeFit can be installed with: `pip install likefit`

In [ ]:
import likefit

## Data

In [ ]:
xdata = np.array([-0.18, -0.14, -0.1, -0.06, -0.02,  0.02,  0.06,  0.1,  0.14])
ydata = np.array([2.243, 2.217, 2.201, 2.175, 2.132, 2.116, 2.083, 2.016, 2.004])
ysigma = np.array([0.008, 0.008, 0.01, 0.009, 0.011, 0.016, 0.018, 0.021, 0.017])

## Fit model 

The fit model has two arguments:
- x: the independent variable
- par: an array with the fit parameters

LikeFit currently supports only a single independent variable x. To ensure the smooth processing by LikeFit, the fit model has to be vectorized in x. This means it has to accept a one-dimensional array containing n values of x and return an array with n values of the fit model. 

In [ ]:
def fit_model(x, par):
    return par[0] + par[1] * x

## Fitter construction

As the independent variable y follows a normal distribution and the fit model is linear in the fit parameters, we select a LinearLeastSquares fitter. We load the data and the fit model in the fitter but do not fit yet.

In [ ]:
fitter = likefit.LinearLeastSquares(xdata, ydata, ysigma, fit_model)

## Fit the data

We now run the fitter. All the results are stored internally within the fitter object.

In [ ]:
fitter.fit()

## Fit results

We print a summary of the fit results

In [ ]:
fitter.print_results()

Different fit results can be retrieved from the fitter. For example, we will get the estimators and their errors and print them in a nice format. 

In [ ]:
estimators = fitter.get_estimators()
errors = fitter.get_errors()
print(f"Intercept: {estimators[0]:.4f} ± {errors[0]:.4f}")
print(f"Slope: {estimators[1]:.3f} ± {errors[1]:.3f}")

We also report the correlation coefficient between the estimators of the two fit parameters. We pick up the correlation coefficient from the correlation matrix.

In [ ]:
correlation_matrix = fitter.get_correlation_matrix()
correlation_coefficient = correlation_matrix[1,0]
print(f"Correlation coefficient: {correlation_coefficient:.2f}")

Finally evaluate the goodness of the fit with a chi-square test.

In [ ]:
degrees_of_freedom = fitter.get_ndof()
print(f"Degrees of freedom: {degrees_of_freedom}")
chi_square = fitter.get_deviance()
print(f"Observed chi-square: {chi_square:.1f}")
pvalue = fitter.get_pvalue()
print(f"Pvalue: {pvalue:.2f}")

We used the get_deviance() method to obtain the chi-square of the fit. The deviance is a statistic that reduces to the known chi-square one for a linear least squares fit. The value is far from zero meaning that the model is consistent with the data

## Fit plot

LikeFit provides a method to display a basic fit plot. In addition to the data and the fit, the plot also shows the error band of the fit. 

In [ ]:
fitter.plot_fit()

LikeFit provides some methods to help you display customised plots. Let us plot the fit and the residuals in the same figure.

In [ ]:
fig = plt.figure(figsize=(6,6))
ax1, ax2 = fig.subplots(2, sharex=True)
ax1.set_ylabel("Fit")
ax2.set_ylabel("Residual")
ax2.set_xlabel("x")

# Plot data 
ax1.errorbar(xdata, ydata, ysigma, ls='none', marker='o', label="Data")

# Plot fit 
xfit = np.linspace(xdata.min(), xdata.max())
yfit = fitter.get_yfit(xfit)
ax1.plot(xfit, yfit, ls='--', label="Fit")

# Plot error band
yfit_error = fitter.get_yfit_error(xfit)
ax1.fill_between(xfit, yfit - yfit_error, yfit + yfit_error, color='tab:orange', alpha=0.2)

ax1.legend()

# Bottom plot
residuals = fitter.get_residuals()
ax2.plot(xdata, residuals, marker='o', ls='')

# Plot horizontal line at y=0
ax2.axhline(ls='--')


## Cost function plot

We finally plot a density map of the cost function. The parameters to display are selected by their index. This is useful when the model has more than 2 parameters. The contours of the 1σ and 2σ confidence regions are shown.

In [ ]:
fitter.plot_confidence_regions(parx_index=0, pary_index=1, xlabel="Intercept", ylabel="Slope")